## Using outer join to select actors

* [Concatenate DataFrames together vertically](#Concatenate-DataFrames-together-vertically)
 * [Simple](#Simple)
 * [With ignore_index parameter](#With-ignore_index-parameter)
 * [With keys parameter](#With-keys-parameter)

In [1]:
import os
from pathlib import Path
import re

import pandas as pd

In [2]:
DATAPATH = Path('data').absolute()

In [3]:
actors_movies = pd.read_csv(DATAPATH / "actors_movies.csv")
actors_movies.head()

,Actor,Title
0,"[Emil Eifrem,Hugo Weaving,Laurence Fishburne,C...",The Matrix
1,"[Hugo Weaving,Laurence Fishburne,Carrie-Anne M...",The Matrix Reloaded
2,"[Hugo Weaving,Laurence Fishburne,Carrie-Anne M...",The Matrix Revolutions
3,"[Al Pacino,Charlize Theron,Keanu Reeves]",The Devil's Advocate
4,"[James Marshall,Kevin Pollak,J.T. Walsh,Aaron ...",A Few Good Men


In [4]:
actors_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Actor   35 non-null     object
 1   Title   35 non-null     object
dtypes: object(2)
memory usage: 688.0+ bytes


In [5]:
actors_movies.Title.str.lower()

0                          the matrix
1                 the matrix reloaded
2              the matrix revolutions
3                the devil's advocate
4                      a few good men
5                             top gun
6                       jerry maguire
7                         stand by me
8                  as good as it gets
9                what dreams may come
10             snow falling on cedars
11                    you've got mail
12               sleepless in seattle
13             joe versus the volcano
14                  that thing you do
15                   the replacements
16                         rescuedawn
17                       the birdcage
18                         unforgiven
19                    johnny mnemonic
20                        cloud atlas
21                  the da vinci code
22                     v for vendetta
23                        speed racer
24                     ninja assassin
25                     the green mile
26          

In [6]:
movies = pd.read_pickle(DATAPATH / "movies.p")
movies.head()

,id,title,popularity,release_date
0,257,Oliver Twist,20.415572,2005-09-23
1,14290,Better Luck Tomorrow,3.877036,2002-01-12
2,38365,Grown Ups,38.864027,2010-06-24
3,9672,Infamous,3.680896,2006-11-16
4,12819,Alpha and Omega,12.300789,2010-09-17


### Exercise 2.3.1 Merging a table to itself

In [7]:
sequels = pd.read_pickle(DATAPATH / "sequels.p")
sequels.head()

,id,title,sequel
0,19995,Avatar,<NA>
1,862,Toy Story,863
2,863,Toy Story 2,10193
3,597,Titanic,<NA>
4,24428,The Avengers,<NA>


In [8]:
sequels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      4803 non-null   Int64 
 1   title   4803 non-null   object
 2   sequel  90 non-null     Int64 
dtypes: Int64(2), object(1)
memory usage: 122.1+ KB


In [9]:
crews = pd.read_pickle(DATAPATH / "crews.p")
crews.head()

,id,department,job,name
0,19995,Editing,Editor,Stephen E. Rivkin
2,19995,Sound,Sound Designer,Christopher Boyes
4,19995,Production,Casting,Mali Finn
6,19995,Directing,Director,James Cameron
7,19995,Writing,Writer,James Cameron


In [10]:
crews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42502 entries, 0 to 129580
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          42502 non-null  int64 
 1   department  42502 non-null  object
 2   job         42502 non-null  object
 3   name        42502 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.6+ MB


In [11]:
# Merge the crews table to itself
crews_self_merged = crews.merge(crews, on='id', how='inner', suffixes=('_dir','_crew'))
crews_self_merged.head()

,id,department_dir,job_dir,name_dir,department_crew,job_crew,name_crew
0,19995,Editing,Editor,Stephen E. Rivkin,Editing,Editor,Stephen E. Rivkin
1,19995,Editing,Editor,Stephen E. Rivkin,Sound,Sound Designer,Christopher Boyes
2,19995,Editing,Editor,Stephen E. Rivkin,Production,Casting,Mali Finn
3,19995,Editing,Editor,Stephen E. Rivkin,Directing,Director,James Cameron
4,19995,Editing,Editor,Stephen E. Rivkin,Writing,Writer,James Cameron


In [12]:
crews_self_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 834194 entries, 0 to 834193
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               834194 non-null  int64 
 1   department_dir   834194 non-null  object
 2   job_dir          834194 non-null  object
 3   name_dir         834194 non-null  object
 4   department_crew  834194 non-null  object
 5   job_crew         834194 non-null  object
 6   name_crew        834194 non-null  object
dtypes: int64(1), object(6)
memory usage: 50.9+ MB


### Self join

**Merging a table to itself can be useful when we want to compare values in a column to other values in the same column.** We will create a table that for each movie will list the movie director and a member of the crew on one row. We have been given a table called crews, which has columns `id`, `job`, and `name`. First, we merge the table to itself using the movie ID. This merge will give us a larger table where for each movie, every job is matched against each other. Then we select only those rows with a director in the left table, and avoid having a row where the director's job is listed in both the left and right tables. This filtering will remove job combinations that aren't with the director.

In [13]:
# Create a boolean index to select the appropriate rows
boolean_filter = ((crews_self_merged['job_dir'] == 'Director') & 
                  (crews_self_merged['job_crew'] != 'Director'))

direct_crews = crews_self_merged[boolean_filter]
direct_crews.head()

,id,department_dir,job_dir,name_dir,department_crew,job_crew,name_crew
156,19995,Directing,Director,James Cameron,Editing,Editor,Stephen E. Rivkin
157,19995,Directing,Director,James Cameron,Sound,Sound Designer,Christopher Boyes
158,19995,Directing,Director,James Cameron,Production,Casting,Mali Finn
160,19995,Directing,Director,James Cameron,Writing,Writer,James Cameron
161,19995,Directing,Director,James Cameron,Art,Set Designer,Richard F. Mays


By merging the table to itself, we compared the value of the director from the jobs column to other values from the jobs column. With the output, we can quickly see different movie directors and the people they worked with in the same movie.

## Section 2.3 Merging on indexes
### Exercise 2.3.1 Index merge for movie ratings

To practice merging on indexes, you will merge `movies` and a table called `ratings` that holds info about movie ratings. Make sure your merge returns all of the rows from the `movies`table and not all the rows of `ratings` table need to be included in the result.

In [14]:
ratings = pd.read_pickle(DATAPATH / "ratings.p")
ratings.head()

,id,vote_average,vote_count
0,19995,7.2,11800.0
1,285,6.9,4500.0
2,206647,6.3,4466.0
3,49026,7.6,9106.0
4,49529,6.1,2124.0


In [15]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   vote_average  4803 non-null   float64
 2   vote_count    4741 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 112.7 KB


In [16]:
# Merge to the movies table the ratings table on the index
movies_ratings = movies.merge(ratings, left_index=True, right_index=True, how='left')
movies_ratings.head()

,id_x,title,popularity,release_date,id_y,vote_average,vote_count
0,257,Oliver Twist,20.415572,2005-09-23,19995,7.2,11800.0
1,14290,Better Luck Tomorrow,3.877036,2002-01-12,285,6.9,4500.0
2,38365,Grown Ups,38.864027,2010-06-24,206647,6.3,4466.0
3,9672,Infamous,3.680896,2006-11-16,49026,7.6,9106.0
4,12819,Alpha and Omega,12.300789,2010-09-17,49529,6.1,2124.0


### Exercise 2.4.2 Do sequels earn more?
<p>It is time to put together many of the aspects that you have learned in this chapter. In this exercise, you'll find out which movie sequels earned the most compared to the original movie. To answer this question, you will merge a modified version of the <code>sequels</code> and <code>financials</code> tables where their index is the movie ID. You will need to choose a merge type that will return all of the rows from the <code>sequels</code> table and not all the rows of <code>financials</code> table need to be included in the result. From there, you will join the resulting table to itself so that you can compare the revenue values of the original movie to the sequel. Next, you will calculate the difference between the two revenues and sort the resulting dataset. </p><p>The <code>sequels</code> and <code>financials</code> tables have been provided.</p>

#### Instructions

<ul><li>With the <code>sequels</code> table on the left, merge to it the <code>financials</code> table on index named <code>id</code>, ensuring that all the rows from the <code>sequels</code> are returned and some rows from the other table may not be returned, Save the results to <code>sequels_fin</code>.</li></ul>

In [17]:
financials = pd.read_pickle(DATAPATH / "financials.p")
financials.head()

,id,budget,revenue
0,19995,237000000,2.787965e+09
1,285,300000000,9.610000e+08
2,206647,245000000,8.806746e+08
3,49026,250000000,1.084939e+09
4,49529,260000000,2.841391e+08


In [18]:
# Merge sequels and financials on index id
sequels_fin = (sequels
               .merge(financials, how='left', left_index=True, right_index=True, suffixes=('', '_y'))
               .drop('id_y', axis=1)
               .set_index('id'))
sequels_fin.head()

,title,sequel,budget,revenue
id,,,,
19995,Avatar,<NA>,237000000.0,2.787965e+09
862,Toy Story,863,300000000.0,9.610000e+08
863,Toy Story 2,10193,245000000.0,8.806746e+08
597,Titanic,<NA>,250000000.0,1.084939e+09
24428,The Avengers,<NA>,260000000.0,2.841391e+08


In [19]:
# Self merge with suffixes as inner join with left on sequel and right on id
orig_seq = sequels_fin.merge(sequels_fin, how='inner', left_on='sequel', 
                             right_on='id', right_index=True,
                             suffixes=('_org','_seq'))
orig_seq.head()

,sequel,title_org,sequel_org,budget_org,revenue_org,title_seq,sequel_seq,budget_seq,revenue_seq
id,,,,,,,,,
862,863,Toy Story,863,300000000.0,9.610000e+08,Toy Story 2,10193,245000000.0,8.806746e+08
863,10193,Toy Story 2,10193,245000000.0,8.806746e+08,Toy Story 3,<NA>,380000000.0,1.045714e+09
675,767,Harry Potter and the Order of the Phoenix,767,200000000.0,7.837663e+08,Harry Potter and the Half-Blood Prince,<NA>,200000000.0,1.215440e+09
121,122,The Lord of the Rings: The Two Towers,122,200000000.0,1.025491e+09,The Lord of the Rings: The Return of the King,<NA>,255000000.0,8.928991e+07
120,121,The Lord of the Rings: The Fellowship of the Ring,121,190000000.0,1.506249e+09,The Lord of the Rings: The Two Towers,122,200000000.0,1.025491e+09


In [20]:
# Add calculation to subtract revenue_org from revenue_seq 
orig_seq['diff'] = orig_seq['revenue_seq'] - orig_seq['revenue_org']

# Select the title_org, title_seq, and diff 
titles_diff = orig_seq[['title_org','title_seq','diff']]
titles_diff.sort_values(by='diff', ascending=False).head()

,title_org,title_seq,diff
id,,,
3049,Ace Ventura: Pet Detective,Ace Ventura: When Nature Calls,1.039243e+09
8966,Twilight,The Twilight Saga: New Moon,6.775028e+08
10138,Iron Man 2,Iron Man 3,6.579765e+08
61791,Rise of the Planet of the Apes,Dawn of the Planet of the Apes,5.895793e+08
331,Jurassic Park III,Jurassic World,5.578606e+08


# 3. Advanced Merging and Concatenating

In this chapter, you'll leverage powerful filtering techniques, including semi-joins and anti-joins. You’ll also learn how to glue DataFrames by vertically combining and using the pandas.concat function to create new datasets. Finally, because data is rarely clean, you’ll also learn how to validate your newly combined data structures.

Tags: semi-join, anti-join.

## Anti-join example

In [37]:
movies_financials = movies.merge(financials, on='id', how='left', indicator=True)
movies_financials.head()

,id,title,popularity,release_date,budget,revenue,_merge
0,257,Oliver Twist,20.415572,2005-09-23,50000000.0,42093706.0,both
1,14290,Better Luck Tomorrow,3.877036,2002-01-12,NaN,NaN,left_only
2,38365,Grown Ups,38.864027,2010-06-24,80000000.0,271430189.0,both
3,9672,Infamous,3.680896,2006-11-16,13000000.0,1151330.0,both
4,12819,Alpha and Omega,12.300789,2010-09-17,20000000.0,39300000.0,both


In [38]:
# Select the vid column where _merge is left_only
id_list = movies_financials.loc[movies_financials['_merge'] == 'left_only', 'id']
id_list

1        14290
7        13785
9        18841
11      158752
15       16186
         ...  
4783     17334
4788     25189
4793    198370
4799     11934
4800     13807
Name: id, Length: 1574, dtype: int64

In [41]:
# Select movies for which there are no financials
movies[movies['id'].isin(id_list)].head()

,id,title,popularity,release_date
1,14290,Better Luck Tomorrow,3.877036,2002-01-12
7,13785,Best in Show,10.982255,2000-09-19
9,18841,The Lost Skeleton of Cadavra,1.680525,2001-09-12
11,158752,Escape from Tomorrow,1.352222,2013-10-11
15,16186,Diary of a Mad Black Woman,3.784535,2005-02-25


As we can observe, there are only movies for which budget and revenue are not available.

In [42]:
movies_financials[movies['id'].isin(id_list)].head()

,id,title,popularity,release_date,budget,revenue,_merge
1,14290,Better Luck Tomorrow,3.877036,2002-01-12,NaN,NaN,left_only
7,13785,Best in Show,10.982255,2000-09-19,NaN,NaN,left_only
9,18841,The Lost Skeleton of Cadavra,1.680525,2001-09-12,NaN,NaN,left_only
11,158752,Escape from Tomorrow,1.352222,2013-10-11,NaN,NaN,left_only
15,16186,Diary of a Mad Black Woman,3.784535,2005-02-25,NaN,NaN,left_only


## Steps of a semi-join

<ol>
    <li>Search if the key column in the left table is in the merged tables using the <code>.isin()</code> method creating a Boolean <code>Series</code>.</li>
    <li>Subset the rows of the left table.</li>
    <li>Merge the left and right tables on key column using an inner-join.</li>
</ol>

<div style="text-align:center;">
    <img src="img/semi_join.png" style="width:500px;">
</div>

## Concatenate DataFrames together vertically

In [47]:
movies.release_date = pd.to_datetime(movies.release_date)

In [51]:
movies_jan = movies[movies.release_date.dt.strftime("%Y-%m") == '2002-01'].head()
movies_jan

,id,title,popularity,release_date
1,14290,Better Luck Tomorrow,3.877036,2002-01-12
39,11022,Narc,8.526635,2002-01-14
194,16653,The Singles Ward,0.920591,2002-01-30
347,99826,The Jimmy Show,0.098351,2002-01-16
459,10229,A Walk to Remember,33.283554,2002-01-25


In [52]:
movies_feb = movies[movies.release_date.dt.strftime("%Y-%m") == '2002-02'].head()
movies_mar = movies[movies.release_date.dt.strftime("%Y-%m") == '2002-03'].head()

### Simple

In [53]:
pd.concat([movies_jan, movies_feb, movies_mar])

,id,title,popularity,release_date
1,14290,Better Luck Tomorrow,3.877036,2002-01-12
39,11022,Narc,8.526635,2002-01-14
194,16653,The Singles Ward,0.920591,2002-01-30
347,99826,The Jimmy Show,0.098351,2002-01-16
459,10229,A Walk to Remember,33.283554,2002-01-25
123,16690,Return to Never Land,15.341196,2002-02-14
183,10592,Hart's War,14.244518,2002-02-15
220,598,City of God,44.356711,2002-02-05
679,10052,Dragonfly,8.927137,2002-02-22
830,102840,Sex With Strangers,0.014406,2002-02-22


### With ignore_index parameter

In [54]:
pd.concat([movies_jan, movies_feb, movies_mar], ignore_index=True)

,id,title,popularity,release_date
0,14290,Better Luck Tomorrow,3.877036,2002-01-12
1,11022,Narc,8.526635,2002-01-14
2,16653,The Singles Ward,0.920591,2002-01-30
3,99826,The Jimmy Show,0.098351,2002-01-16
4,10229,A Walk to Remember,33.283554,2002-01-25
5,16690,Return to Never Land,15.341196,2002-02-14
6,10592,Hart's War,14.244518,2002-02-15
7,598,City of God,44.356711,2002-02-05
8,10052,Dragonfly,8.927137,2002-02-22
9,102840,Sex With Strangers,0.014406,2002-02-22


### With keys parameter

In [56]:
pd.concat([movies_jan, movies_feb, movies_mar], ignore_index=False, keys=['jan', 'feb', 'mar'])

id                          title  popularity release_date
jan 1      14290           Better Luck Tomorrow    3.877036   2002-01-12
    39     11022                           Narc    8.526635   2002-01-14
    194    16653               The Singles Ward    0.920591   2002-01-30
    347    99826                 The Jimmy Show    0.098351   2002-01-16
    459    10229             A Walk to Remember   33.283554   2002-01-25
feb 123    16690           Return to Never Land   15.341196   2002-02-14
    183    10592                     Hart's War   14.244518   2002-02-15
    220      598                    City of God   44.356711   2002-02-05
    679    10052                      Dragonfly    8.927137   2002-02-22
    830   102840             Sex With Strangers    0.014406   2002-02-22
mar 60      2752          40 Days and 40 Nights   14.181360   2002-03-01
    524    11452  National Lampoon’s Van Wilder   35.108498   2002-03-29
    1568    4547                     Panic Room   53.043124   2002-03-29
    1724   39541                  Stolen Summer    0.870660   2002-03-22
    1798    2135               The Time Machine   25.978555   2002-03-04